In [1]:
import pandas as pd

In [5]:
import os
os.chdir("../Sup_Table_S4")

In [2]:
PHYLA = ['Annelida', 'Arthropoda', 'Chordata', 'Cnidaria', 'Echinodermata', 'Mollusca', 'Nematoda', 'Platyhelminthes']

In [3]:
def load_preds(file_name):
    preds = pd.read_csv(file_name, sep=',')
    preds.columns = ['header', 'rank', 'pred', 'score']
    preds['sid'] = preds['header'].apply(lambda x: x.split(' |')[0])
    
    return preds

In [4]:
def get_FP(preds, sp_info, cutoff):
    df = pd.merge(left=preds, right=sp_info, left_on='sid', right_on='sid', how='inner')
    excluded = df.query("species == 'excluded'")
    
    total = excluded.query("rank == 'species'")
    FP = total.query("score > @cutoff")
    
    return FP, total

In [6]:
tmp = {'DeepCOI': {}, 'RDP': {}, 'BLAST': {}}

for phylum in PHYLA:
    sp_info = pd.read_csv(f"../../data/testset_labels/{phylum}.test.labels.txt", sep=',')
    
    coi = load_preds(f"{phylum}.preds.csv")
    FP_coi, total_coi = get_FP(coi, sp_info, 0.9)
    tmp['DeepCOI'][phylum] = FP_coi.shape[0] / total_coi.shape[0]
    
    rdp = load_preds(f"{phylum}.rdp.csv")
    FP_rdp, total_rdp = get_FP(rdp, sp_info, 0.8)
    tmp['RDP'][phylum] = FP_rdp.shape[0] / total_rdp.shape[0]
    
    blast = load_preds(f"{phylum}.blast.csv")
    FP_blast, total_blast = get_FP(blast, sp_info, 94)
    tmp['BLAST'][phylum] = FP_blast.shape[0] / total_blast.shape[0]
    
df = pd.DataFrame(tmp)

In [7]:
df

,DeepCOI,RDP,BLAST
Annelida,0.177298,0.517010,0.195289
Arthropoda,0.115794,0.308865,0.344482
Chordata,0.259539,0.584183,0.409348
Cnidaria,0.414410,0.735309,0.608074
Echinodermata,0.212784,0.590734,0.293436
Mollusca,0.277985,0.551507,0.336932
Nematoda,0.144968,0.543860,0.183749
Platyhelminthes,0.157175,0.457099,0.211845
